# 04 - Injury Features & Dataset Splitting

Bu notebook, Ekin'in sorumluluğundaki görevleri gerçekleştirir:

1. **Injury Data Processing**: Raw injury verilerini temizle ve birleştir
2. **Injury Feature Generation**: Maç bazlı injury feature'larını üret
3. **Final Dataset**: Core features + injury features ile final dataset oluştur
4. **Train/Val/Test Split**: Zaman bazlı split yap

## Input Dosyalar
- `data_interim/games_with_core_features.csv` (İbrahim'den)
- `data_raw/injury_reports_raw/` (Raw injury PDF'lerden parse edilmiş CSV'ler)
- `data_raw/nbastuffer_2025_2026_player_stats_raw.csv` (Oyuncu dakika bilgileri)

## Output Dosyalar
- `data_interim/injury_reports_clean.csv` (Temizlenmiş injury verileri)
- `data_processed/games_with_all_features.csv` (Core + injury features)
- `data_processed/train_set.csv`
- `data_processed/val_set.csv`
- `data_processed/test_set.csv`


## 1. Setup & Imports


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Proje root dizinini bul
project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Çalışma dizinini proje root'una ayarla
os.chdir(project_root)

print(f"Proje root: {project_root}")
print(f"Çalışma dizini: {os.getcwd()}")


Proje root: c:\Users\user\Desktop\ann odev
Çalışma dizini: c:\Users\user\Desktop\ann odev


In [2]:
# Modülleri import et
from src.features.injury_features import (
    load_and_clean_injury_reports,
    load_player_minutes,
    add_injury_features,
    build_injury_features
)

from src.data.split_dataset import (
    analyze_date_range,
    split_dataset_by_time,
    split_dataset_random,
    save_splits,
    print_split_stats,
    print_split_stats_random,
    validate_splits,
    create_time_based_split,
    create_random_split,
    suggest_split_dates
)

print("Modüller yüklendi!")


Modüller yüklendi!


## 2. Core Features Dosyasını Kontrol Et

İbrahim'in build_features.py pipeline'ının çıktısını kontrol edelim. Eğer yoksa, oluşturalım.


In [3]:
# İlk olarak İbrahim'in pipeline'ını çalıştırıp core features'ı üretelim
from src.features.build_features import build_model_dataset

# master_merged.csv yoksa, önce onu oluşturmamız gerekiyor
master_csv = Path("data_processed/master_merged.csv")

if not master_csv.exists():
    print("master_merged.csv bulunamadı!")
    print("Önce 02_clean_merge.ipynb notebook'unu çalıştırın.")
else:
    print(f"master_merged.csv mevcut: {master_csv}")
    
    # Core features'ı oluştur
    core_features_path = Path("data_interim/games_with_core_features.csv")
    if not core_features_path.exists():
        print("\nCore features oluşturuluyor...")
        build_model_dataset(
            master_csv=master_csv,
            output_csv="data_processed/model_dataset.csv",
            write_interim=True
        )
    else:
        print(f"Core features mevcut: {core_features_path}")


master_merged.csv mevcut: data_processed\master_merged.csv

Core features oluşturuluyor...
[OK] games_with_core_features.csv written -> data_interim\games_with_core_features.csv (17832 rows, 219 cols)
[OK] model_dataset.csv written -> data_processed\model_dataset.csv (17820 rows, 94 cols)


In [4]:
# Core features dosyasını yükle ve incele
core_features_path = Path("data_interim/games_with_core_features.csv")

if core_features_path.exists():
    core_df = pd.read_csv(core_features_path, low_memory=False)
    
    print(f"Core Features:")
    print(f"  Satır sayısı: {len(core_df):,}")
    print(f"  Kolon sayısı: {len(core_df.columns)}")
    
    print(f"\nKolonlar:")
    for i, col in enumerate(core_df.columns):
        print(f"  {i+1}. {col}")
    
    print(f"\nİlk 3 satır:")
    display(core_df.head(3))
    
    # Tarih aralığını kontrol et
    if 'game_date' in core_df.columns:
        core_df['game_date'] = pd.to_datetime(core_df['game_date'])
        print(f"\nTarih aralığı:")
        print(f"  Min: {core_df['game_date'].min()}")
        print(f"  Max: {core_df['game_date'].max()}")
else:
    print("Core features dosyası bulunamadı!")


Core Features:
  Satır sayısı: 17,832
  Kolon sayısı: 219

Kolonlar:
  1. gameId
  2. game_date
  3. season_year
  4. home_team
  5. away_team
  6. home_team_code
  7. away_team_code
  8. season_type
  9. matchup
  10. RANK
  11. home_team_CONF
  12. home_team_DIVISION
  13. home_team_GP
  14. home_team_PPG
  15. home_team_oPPG
  16. home_team_pDIFF
  17. home_team_PACE
  18. home_team_oEFF
  19. home_team_dEFF
  20. home_team_eDIFF
  21. home_team_SoS
  22. home_team_rSoS
  23. home_team_SAR
  24. home_team_CONS
  25. home_team_A4F
  26. home_team_W
  27. home_team_L
  28. home_team_WIN%
  29. home_team_eWIN%
  30. home_team_pWIN%
  31. home_team_ACH
  32. home_team_STRK
  33. RANK_away_stats
  34. away_team_CONF
  35. away_team_DIVISION
  36. away_team_GP
  37. away_team_PPG
  38. away_team_oPPG
  39. away_team_pDIFF
  40. away_team_PACE
  41. away_team_oEFF
  42. away_team_dEFF
  43. away_team_eDIFF
  44. away_team_SoS
  45. away_team_rSoS
  46. away_team_SAR
  47. away_team_CONS
  

,gameId,game_date,season_year,home_team,away_team,home_team_code,away_team_code,season_type,matchup,RANK,...,diff_schedule_ALL B2B,diff_schedule_TOTAL B2B ON THE ROAD,diff_schedule_TOTAL B2B AT HOME,diff_schedule_3IN4,diff_schedule_1-DAY REST,diff_schedule_2-DAYS REST,diff_schedule_3+DAYS REST,diff_schedule_REST ADVANTAGE,diff_schedule_REST DISADVANTAGE,diff_schedule_BOTH TEAMS RESTED or NO-REST
0,21000001,2010-10-26,2010-11,Boston Celtics,Miami Heat,BOS,MIA,regular,BOS vs. MIA,NaN,...,-2,1,-3,0,5,-5,2,1,-2,1
1,21000002,2010-10-26,2010-11,Portland Trail Blazers,Phoenix Suns,POR,PHX,regular,PHX @ POR,NaN,...,-1,2,-3,-3,5,-1,0,3,-3,0
2,21000003,2010-10-26,2010-11,Los Angeles Lakers,Houston Rockets,LAL,HOU,regular,HOU @ LAL,NaN,...,0,-2,2,1,-1,0,0,1,1,-2



Tarih aralığı:
  Min: 2010-10-26 00:00:00
  Max: 2025-11-15 00:00:00


## 3. Injury Data Processing

Raw injury verilerini yükle, temizle ve birleştir.


In [5]:
# Raw injury klasörünü kontrol et
injury_raw_dir = Path("data_raw/injury_reports_raw")

if injury_raw_dir.exists():
    files = list(injury_raw_dir.glob("*"))
    print(f"Injury raw dosyaları ({len(files)} dosya):")
    for f in files:
        print(f"  - {f.name}")
else:
    print("Injury raw klasörü bulunamadı!")


Injury raw klasörü bulunamadı!


In [6]:
# Injury verilerini yükle ve temizle
# NOT: Injury verisi sınırlı (sadece 1 günlük), bu yüzden opsiyonel
try:
    injury_df = load_and_clean_injury_reports(
        raw_dir="data_raw/injury_reports_raw/",
        output_path="data_interim/injury_reports_clean.csv"
    )
except Exception as e:
    print(f"⚠️ Injury verisi yüklenirken hata: {e}")
    injury_df = pd.DataFrame()

if len(injury_df) > 0:
    print(f"\n✅ Temizlenmiş injury verileri:")
    print(f"  Satır sayısı: {len(injury_df)}")
    print(f"\nKolonlar: {list(injury_df.columns)}")
    print(f"\nİlk 10 satır:")
    display(injury_df.head(10))

    print(f"\nStatus dağılımı:")
    print(injury_df['status'].value_counts())
else:
    print("⚠️ Injury verisi bulunamadı veya parse edilemedi.")
    print("   → İbrahim'in yaklaşımı: Injury olmadan devam edilecek (inference-time'da kullanılacak)")


⚠️ Injury klasörü bulunamadı: data_raw\injury_reports_raw
⚠️ Injury verisi bulunamadı veya parse edilemedi.
   → İbrahim'in yaklaşımı: Injury olmadan devam edilecek (inference-time'da kullanılacak)


## 4. Player Minutes Data


In [7]:
# Player minutes verilerini yükle (opsiyonel - injury için gerekli)
try:
    player_minutes_df = load_player_minutes(
        player_stats_path="data_raw/nbastuffer_2025_2026_player_stats_raw.csv"
    )
except Exception as e:
    print(f"⚠️ Player minutes yüklenirken hata: {e}")
    player_minutes_df = pd.DataFrame()

if len(player_minutes_df) > 0:
    print(f"\n✅ Player minutes verileri:")
    print(f"  Oyuncu sayısı: {len(player_minutes_df)}")

    print(f"\nEn çok oynayan oyuncular:")
    top_players = player_minutes_df.nlargest(15, 'avg_minutes_per_game')
    display(top_players)

    print(f"\nDakika dağılımı:")
    print(player_minutes_df['avg_minutes_per_game'].describe())

    # Key players (25+ dk)
    key_players = player_minutes_df[player_minutes_df['avg_minutes_per_game'] >= 25]
    print(f"\nKey players (25+ dk): {len(key_players)} oyuncu")
else:
    print("⚠️ Player minutes verisi bulunamadı - Injury features için gerekli değil")


⚠️ Player stats dosyası bulunamadı: data_raw\nbastuffer_2025_2026_player_stats_raw.csv
⚠️ Player minutes verisi bulunamadı - Injury features için gerekli değil


## 5. Injury Features Generation

Core features'a injury feature'larını ekle.


In [8]:
# Injury features ekle (veya placeholder kullan)
if core_features_path.exists():
    # Injury verisi varsa ve yeterliyse kullan
    if len(injury_df) > 0 and 'player_minutes_df' in dir() and len(player_minutes_df) > 0:
        try:
            games_with_injury = add_injury_features(
                games_df=core_df,
                injury_df=injury_df,
                player_minutes_df=player_minutes_df,
                key_player_minutes_threshold=25.0
            )
            print(f"\n✅ Injury features eklendi:")
            new_cols = [col for col in games_with_injury.columns if col not in core_df.columns]
            print(f"  Yeni kolonlar: {new_cols}")
        except Exception as e:
            print(f"⚠️ Injury features eklenirken hata: {e}")
            games_with_injury = core_df.copy()
    else:
        # İbrahim'in yaklaşımı: Injury verisi yetersiz, placeholder kullan
        print("⚠️ Yeterli injury verisi yok - İbrahim'in yaklaşımı uygulanıyor:")
        print("   → Injury feature'ları 0 olarak ayarlanacak")
        print("   → Model sadece core features ile eğitilecek")
        print("   → Injury, inference-time'da ayrı kullanılacak")
        
        games_with_injury = core_df.copy()
        # Placeholder injury kolonları (hepsi 0)
        games_with_injury['injury_count_home'] = 0
        games_with_injury['injury_count_away'] = 0
        games_with_injury['expected_minutes_lost_home'] = 0.0
        games_with_injury['expected_minutes_lost_away'] = 0.0
        games_with_injury['any_key_player_out_home'] = 0
        games_with_injury['any_key_player_out_away'] = 0

    print(f"\n📊 Final dataset:")
    print(f"  Satır sayısı: {len(games_with_injury):,}")
    print(f"  Kolon sayısı: {len(games_with_injury.columns)}")
else:
    print("❌ Core features dosyası bulunamadı!")


⚠️ Yeterli injury verisi yok - İbrahim'in yaklaşımı uygulanıyor:
   → Injury feature'ları 0 olarak ayarlanacak
   → Model sadece core features ile eğitilecek
   → Injury, inference-time'da ayrı kullanılacak

📊 Final dataset:
  Satır sayısı: 17,832
  Kolon sayısı: 225


In [9]:
# Final dataset'i kaydet
output_path = Path("data_processed/games_with_all_features.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

games_with_injury.to_csv(output_path, index=False)
print(f"Final dataset kaydedildi: {output_path}")
print(f"   {len(games_with_injury):,} satır, {len(games_with_injury.columns)} kolon")


Final dataset kaydedildi: data_processed\games_with_all_features.csv
   17,832 satır, 225 kolon


## 6. Dataset Splitting (Random Shuffle)

Rastgele karıştırmalı (random shuffle) train/val/test split yap.

**Neden Random Split?**
- Basketbolun oyun yapısı yıllar içinde değiştiği için, sadece eski verilerle eğitilip güncel maçları tahmin etmek (distribution drift) performans kaybına yol açabilir.
- Random split ile model her yıldan (özellikle güncel yıllardan da) veri görerek eğitilmiş olur.

**Split Oranları:**
- Train: %70
- Validation: %15
- Test: %15


In [10]:
# Tarih aralığını analiz et
print("Tarih Aralığı Analizi:")
stats = analyze_date_range(games_with_injury, date_col='game_date')

print(f"\nMin tarih: {stats['min_date']}")
print(f"Max tarih: {stats['max_date']}")
print(f"Toplam gün: {stats['date_range_days']}")
print(f"Toplam maç: {stats['total_games']:,}")

print(f"\nYıllara göre maç sayısı:")
for year, count in sorted(stats['games_per_year'].items()):
    print(f"  {year}: {count:,}")


Tarih Aralığı Analizi:

Min tarih: 2010-10-26 00:00:00
Max tarih: 2025-11-15 00:00:00
Toplam gün: 5499
Toplam maç: 17,832

Yıllara göre maç sayısı:
  2010: 482
  2011: 885
  2012: 1,474
  2013: 1,324
  2014: 1,334
  2015: 1,319
  2016: 1,333
  2017: 1,347
  2018: 1,316
  2019: 1,267
  2020: 706
  2021: 1,625
  2022: 1,336
  2023: 1,248
  2024: 824
  2025: 12


In [11]:
# Random Split Konfigürasyonu
TRAIN_RATIO = 0.70  # %70 Train
VAL_RATIO = 0.15    # %15 Validation  
TEST_RATIO = 0.15   # %15 Test
RANDOM_STATE = 30   # Reproducibility için

print(f"Random Split Konfigürasyonu:")
print(f"  Train:  {TRAIN_RATIO:.0%}")
print(f"  Val:    {VAL_RATIO:.0%}")
print(f"  Test:   {TEST_RATIO:.0%}")
print(f"  Random State: {RANDOM_STATE}")

# Not: Eski zaman bazlı split için suggest_split_dates kullanılabilir:
# suggested_train_end, suggested_val_end = suggest_split_dates(
#     games_with_injury,
#     date_col='game_date',
#     train_ratio=0.70,
#     val_ratio=0.15
# )


Random Split Konfigürasyonu:
  Train:  70%
  Val:    15%
  Test:   15%
  Random State: 30


In [12]:
# Veri özeti
print(f"Toplam maç sayısı: {len(games_with_injury):,}")
print(f"\nBeklenen split sonuçları:")
print(f"  Train: ~{int(len(games_with_injury) * TRAIN_RATIO):,} maç")
print(f"  Val:   ~{int(len(games_with_injury) * VAL_RATIO):,} maç")
print(f"  Test:  ~{int(len(games_with_injury) * TEST_RATIO):,} maç")

# Not: Eski zaman bazlı split için:
# TRAIN_END = "2021-07-01"
# VAL_END = "2023-07-01"
# TEST_END = None


Toplam maç sayısı: 17,832

Beklenen split sonuçları:
  Train: ~12,482 maç
  Val:   ~2,674 maç
  Test:  ~2,674 maç


In [13]:
# Random Split yap
train_df, val_df, test_df = split_dataset_random(
    games_with_injury,
    train_ratio=TRAIN_RATIO,
    val_ratio=VAL_RATIO,
    test_ratio=TEST_RATIO,
    random_state=RANDOM_STATE
)

# İstatistikleri göster (yıl dağılımı ile)
print_split_stats_random(train_df, val_df, test_df, date_col='game_date', label_col='home_team_win')

# Not: Eski zaman bazlı split için:
# train_df, val_df, test_df = split_dataset_by_time(
#     games_with_injury,
#     train_end=TRAIN_END,
#     val_end=VAL_END,
#     test_end=TEST_END,
#     date_col='game_date'
# )
# print_split_stats(train_df, val_df, test_df, date_col='game_date', label_col='home_team_win')



RANDOM SPLIT İSTATİSTİKLERİ

Train:
  Maç sayısı: 12,482 (70.0%)
  Tarih aralığı: 2010-10-26 - 2025-11-12
  Yıl dağılımı: 2010: 342, 2011: 627, 2012: 1043, 2013: 937, 2014: 958, 2015: 917, 2016: 928, 2017: 941, 2018: 927, 2019: 893, 2020: 492, 2021: 1126, 2022: 918, 2023: 867, 2024: 557, 2025: 9
  Home win rate: 57.6%

Val:
  Maç sayısı: 2,675 (15.0%)
  Tarih aralığı: 2010-10-26 - 2025-11-15
  Yıl dağılımı: 2010: 62, 2011: 143, 2012: 216, 2013: 183, 2014: 187, 2015: 209, 2016: 204, 2017: 197, 2018: 173, 2019: 184, 2020: 112, 2021: 264, 2022: 222, 2023: 186, 2024: 131, 2025: 2
  Home win rate: 56.9%

Test:
  Maç sayısı: 2,675 (15.0%)
  Tarih aralığı: 2010-10-27 - 2025-10-23
  Yıl dağılımı: 2010: 78, 2011: 115, 2012: 215, 2013: 204, 2014: 189, 2015: 193, 2016: 201, 2017: 209, 2018: 216, 2019: 190, 2020: 102, 2021: 235, 2022: 196, 2023: 195, 2024: 136, 2025: 1
  Home win rate: 55.9%



In [14]:
# Validation kontrolleri
required_cols = ['home_team_win', 'score_diff', 'game_date', 'home_team', 'away_team']

is_valid = validate_splits(
    train_df, val_df, test_df,
    date_col='game_date',
    required_cols=required_cols,
    check_date_overlap=False  # Random split için tarih overlap kontrolü kapalı
)



✅ Tüm validation kontrolleri başarılı!


In [15]:
# Split dosyalarını kaydet
paths = save_splits(
    train_df, val_df, test_df,
    output_dir='data_processed/',
    prefix=''
)

print("\nKaydedilen dosyalar:")
for name, path in paths.items():
    df = pd.read_csv(path)
    print(f"  {name}: {path} ({len(df):,} satır)")



Kaydedilen dosyalar:
  train: data_processed\train_set.csv (12,482 satır)
  val: data_processed\val_set.csv (2,675 satır)
  test: data_processed\test_set.csv (2,675 satır)


## 7. Final Verification


In [16]:
# Tüm çıktı dosyalarını kontrol et
output_files = [
    "data_interim/injury_reports_clean.csv",
    "data_processed/games_with_all_features.csv",
    "data_processed/train_set.csv",
    "data_processed/val_set.csv",
    "data_processed/test_set.csv"
]

print("=" * 60)
print("ÇIKTI DOSYALARI KONTROLÜ")
print("=" * 60)

for file_path in output_files:
    path = Path(file_path)
    if path.exists():
        df = pd.read_csv(path)
        print(f"\n[OK] {file_path}")
        print(f"     Satır: {len(df):,}, Kolon: {len(df.columns)}")
    else:
        print(f"\n[X] {file_path} bulunamadı!")


ÇIKTI DOSYALARI KONTROLÜ

[X] data_interim/injury_reports_clean.csv bulunamadı!

[OK] data_processed/games_with_all_features.csv
     Satır: 17,832, Kolon: 225

[OK] data_processed/train_set.csv
     Satır: 12,482, Kolon: 225

[OK] data_processed/val_set.csv
     Satır: 2,675, Kolon: 225

[OK] data_processed/test_set.csv
     Satır: 2,675, Kolon: 225


In [17]:
# Train/Val/Test overlap kontrolü
print("\n" + "=" * 60)
print("OVERLAP KONTROLÜ")
print("=" * 60)

train_dates = pd.to_datetime(train_df['game_date'])
val_dates = pd.to_datetime(val_df['game_date'])
test_dates = pd.to_datetime(test_df['game_date'])

print(f"\nTrain: {train_dates.min()} - {train_dates.max()}")
print(f"Val:   {val_dates.min()} - {val_dates.max()}")
print(f"Test:  {test_dates.min()} - {test_dates.max()}")

# Overlap var mı?
train_val_overlap = train_dates.max() >= val_dates.min()
val_test_overlap = val_dates.max() >= test_dates.min()

if train_val_overlap:
    print("\n[X] Train-Val overlap var!")
else:
    print("\n[OK] Train-Val overlap yok")

if val_test_overlap:
    print("[X] Val-Test overlap var!")
else:
    print("[OK] Val-Test overlap yok")



OVERLAP KONTROLÜ

Train: 2010-10-26 00:00:00 - 2025-11-12 00:00:00
Val:   2010-10-26 00:00:00 - 2025-11-15 00:00:00
Test:  2010-10-27 00:00:00 - 2025-10-23 00:00:00

[X] Train-Val overlap var!
[X] Val-Test overlap var!


In [18]:
# Label dağılımı kontrolü
print("\n" + "=" * 60)
print("LABEL DAĞILIMI")
print("=" * 60)

for name, df in [("Train", train_df), ("Val", val_df), ("Test", test_df)]:
    if 'home_team_win' in df.columns:
        win_rate = df['home_team_win'].mean() * 100
        print(f"\n{name}: Home win rate = {win_rate:.1f}%")
    
    if 'score_diff' in df.columns:
        avg_diff = df['score_diff'].mean()
        std_diff = df['score_diff'].std()
        print(f"       Score diff = {avg_diff:.2f} +/- {std_diff:.2f}")



LABEL DAĞILIMI

Train: Home win rate = 57.6%
       Score diff = 2.46 +/- 13.99

Val: Home win rate = 56.9%
       Score diff = 2.35 +/- 13.98

Test: Home win rate = 55.9%
       Score diff = 1.76 +/- 14.20


## Summary

Bu notebook'un ürettiği dosyalar:

| Dosya | Açıklama |
|-------|----------|
| `data_interim/injury_reports_clean.csv` | Temizlenmiş injury verileri |
| `data_processed/games_with_all_features.csv` | Core + Injury features |
| `data_processed/train_set.csv` | Training seti |
| `data_processed/val_set.csv` | Validation seti |
| `data_processed/test_set.csv` | Test seti |

İbrahim bu dosyaları model eğitiminde kullanabilir:
- `train_set.csv`: Model eğitimi için
- `val_set.csv`: Hiperparametre optimizasyonu için
- `test_set.csv`: Final değerlendirme için


In [19]:
print("\n" + "=" * 60)
print("TÜM İŞLEMLER TAMAMLANDI!")
print("=" * 60)
print("\nİbrahim için hazır dosyalar:")
print("  - data_processed/train_set.csv")
print("  - data_processed/val_set.csv")
print("  - data_processed/test_set.csv")



TÜM İŞLEMLER TAMAMLANDI!

İbrahim için hazır dosyalar:
  - data_processed/train_set.csv
  - data_processed/val_set.csv
  - data_processed/test_set.csv
